In [1]:
import numpy as np
import pickle
import pandas as pd
import re
import string

pd.set_option('display.max_rows', 100)

In [ ]:
catboost_result = pd.read_csv('Datasets/test_predictions_log_and_usd.csv', header=0, index_col=None)

In [19]:
quantile_cutoffs = [8.069029328774958, 10.261895993101012, 13.001625835928326, 9999]
quantile_names = ['1 LOW','2 MID-LOW','3 MID-HIGH','4 HIGH']


for q, r in zip(reversed(quantile_cutoffs), reversed(quantile_names)):
    catboost_result.loc[catboost_result['Log Price True']<=q, 'RANGE'] = r

catboost_result['Covered_in_pm_20%'] = ((catboost_result['Price True USD']/catboost_result['Price Predicted USD']-1).abs()<=0.2).astype(int)
catboost_result['Covered_in_pm_50%'] = ((catboost_result['Price True USD']/catboost_result['Price Predicted USD']-1).abs()<=0.5).astype(int)

catboost_result.groupby('RANGE')[['Covered_in_pm_20%','Covered_in_pm_50%']].mean()

,Covered_in_pm_20%,Covered_in_pm_50%
RANGE,,
1 LOW,0.155620,0.380403
2 MID-LOW,0.189411,0.495899
3 MID-HIGH,0.137217,0.353398
4 HIGH,0.077748,0.179625


In [ ]:
result=pd.read_csv('Datasets/Price_Estimates.csv', header=0, index_col=None)

In [7]:
result['Estimates_Mid'] = (result['Price_Estimate_Upper'] + result['Price_Estimate_Lower'])/2
result['Estimates_Range'] = result['Price_Estimate_Upper'] - result['Price_Estimate_Lower']
result['Estimates_Range_USD'] = result['Estimates_Range']*(result['Price Sold USD']/result['Price_Ori_Currency'])
result['Estimates_Range_Perc'] = result['Estimates_Range']/(2*result['Estimates_Mid'])
result['Estimates_Covers_True'] = ((result['Price_Ori_Currency']>=result['Price_Estimate_Lower'])&(result['Price_Ori_Currency']<=result['Price_Estimate_Upper'])).astype(int)

In [10]:
quantile_cutoffs=[8.069029328774958, 10.261895993101012, 13.001625835928326, 9999]
quantile_names = ['1 LOW','2 MID-LOW','3 MID-HIGH','4 HIGH']

for q, r in zip(reversed(quantile_cutoffs), reversed(quantile_names)):
    result.loc[np.log(result['Price Sold USD'])<=q, 'RANGE'] = r

In [12]:
pd.options.display.float_format = '{:,.2f}'.format

result[result['Estimates_Range'].notnull()].groupby('RANGE').apply(lambda x: pd.Series(
    {
        # 'No. Work': x['Price Sold USD'].count(),
        'Average Price (USD)': x['Price Sold USD'].mean(),
        # 'Estimates Available':x['Estimates_Range'].notnull().mean(),
        # 'Average Estimates Range (Original Currency)': x['Estimates_Range'].mean(skipna=True),
        'Average Estimates Range (USD)': x['Estimates_Range_USD'].mean(skipna=True),
        'Average Estimates Range (%)': x['Estimates_Range_Perc'].mean(skipna=True)*100,
        '%Actual Falls in Estimates': x['Estimates_Covers_True'].mean(skipna=True)*100,
        '%Actual Covered with Range < 20%': ((x['Estimates_Covers_True']==1)&(x['Estimates_Range_Perc']<=0.2)).mean(skipna=True)*100,
        '%Actual Covered with Mid +- 20%': ((x['Price_Ori_Currency']/x['Estimates_Mid']-1).abs()<=0.2).mean(skipna=True)*100
    }
))

,Average Price (USD),Average Estimates Range (USD),Average Estimates Range (%),%Actual Falls in Estimates,%Actual Covered with Range < 20%,%Actual Covered with Mid +- 20%
RANGE,,,,,,
1 LOW,"1,862.66",818.32,20.56,43.22,31.16,41.47
2 MID-LOW,"12,443.01","3,665.85",18.97,37.57,29.99,39.94
3 MID-HIGH,"118,432.27","32,386.50",18.57,33.44,25.69,34.84
4 HIGH,"3,230,704.89","1,010,067.73",18.60,41.70,31.89,43.33
